# 📊 Calibration Dashboard

Interactive dashboard за мониторинг на калибрацията на Football AI модела

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import json
from datetime import datetime, timedelta
import os
import sys

# Добавя path към проекта
sys.path.append('..')

from monitoring.calibration_metrics import CalibrationMonitor, evaluate_calibration
from monitoring.adaptive_tuning import AdaptiveTuner

# Настройки за визуализация
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Calibration Dashboard loaded!")

## 🔍 Load Recent Data

In [ ]:
# Инициализира мониторинг компонентите
monitor = CalibrationMonitor()
tuner = AdaptiveTuner()

# Зарежда скорошни данни
print("🔄 Loading recent predictions...")
df = monitor.load_recent_predictions(days=30, max_samples=5000)

if len(df) == 0:
    print("❌ No prediction data found!")
else:
    print(f"✅ Loaded {len(df)} predictions")
    
    # Изчислява реалните outcomes
    df = monitor.calculate_actual_outcomes(df)
    
    # Филтрира само мачове с резултати
    df_with_results = df.dropna(subset=['actual_home_score', 'actual_away_score'])
    
    print(f"📈 {len(df_with_results)} matches with results available")
    
    # Показва основна информация
    print(f"📅 Date range: {df['prediction_date'].min().date()} to {df['prediction_date'].max().date()}")
    print(f"🏆 Leagues: {df['league'].value_counts().to_dict()}")

## 📊 Current Calibration Metrics

In [ ]:
if len(df_with_results) > 0:
    # Генерира калибрационен отчет
    report = monitor.generate_calibration_report(days=30)
    
    if 'error' not in report:
        print("📊 CURRENT CALIBRATION METRICS")
        print("=" * 50)
        
        # OU2.5 Metrics
        if 'ou25' in report:
            ou25 = report['ou25']
            print(f"🎯 OVER/UNDER 2.5:")
            print(f"   ECE: {ou25['ece']:.4f} ({'✅ Good' if ou25['ece'] < 0.07 else '⚠️ High'})")
            print(f"   Brier Score: {ou25['brier_score']:.4f} ({'✅ Good' if ou25['brier_score'] < 0.20 else '⚠️ High'})")
            print(f"   Log Loss: {ou25['log_loss']:.4f}")
            print(f"   Accuracy: {ou25['accuracy']:.3f}")
            print()
        
        # BTTS Metrics
        if 'btts' in report:
            btts = report['btts']
            print(f"⚽ BOTH TEAMS TO SCORE:")
            print(f"   ECE: {btts['ece']:.4f} ({'✅ Good' if btts['ece'] < 0.07 else '⚠️ High'})")
            print(f"   Brier Score: {btts['brier_score']:.4f} ({'✅ Good' if btts['brier_score'] < 0.20 else '⚠️ High'})")
            print(f"   Log Loss: {btts['log_loss']:.4f}")
            print(f"   Accuracy: {btts['accuracy']:.3f}")
            print()
        
        # 1X2 Metrics
        if '1x2' in report and 'overall' in report['1x2']:
            x12_overall = report['1x2']['overall']
            print(f"🏆 1X2 OVERALL:")
            print(f"   Brier Score: {x12_overall['brier_score']:.4f} ({'✅ Good' if x12_overall['brier_score'] < 0.20 else '⚠️ High'})")
            print(f"   Log Loss: {x12_overall['log_loss']:.4f}")
            print(f"   Accuracy: {x12_overall['accuracy']:.3f}")
            
            # Per-class metrics
            for outcome in ['home_win', 'draw', 'away_win']:
                if outcome in report['1x2']:
                    metrics = report['1x2'][outcome]
                    print(f"   {outcome.title()}: ECE={metrics['ece']:.4f}, Brier={metrics['brier_score']:.4f}")
    else:
        print(f"❌ Error generating report: {report['error']}")
else:
    print("❌ No data with results available for calibration analysis")

## 📈 Reliability Curves

In [ ]:
if len(df_with_results) > 0 and 'error' not in report:
    # Създава reliability plots
    fig = monitor.plot_reliability_curves(report)
    plt.show()
    
    # Запазва plot-а
    plot_file = f"reports/calibration/reliability_curves_{datetime.now().strftime('%Y%m%d')}.png"
    os.makedirs(os.path.dirname(plot_file), exist_ok=True)
    fig.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"💾 Reliability curves saved: {plot_file}")

## 📊 Interactive Calibration Trends

In [ ]:
if len(df_with_results) > 0:
    # Подготвя данни за trend анализ
    df_with_results['prediction_date'] = pd.to_datetime(df_with_results['prediction_date'])
    df_with_results['week'] = df_with_results['prediction_date'].dt.to_period('W')
    
    # Изчислява седмични метрики
    weekly_metrics = []
    
    for week in df_with_results['week'].unique():
        week_data = df_with_results[df_with_results['week'] == week]
        
        if len(week_data) >= 10:  # Минимум 10 мача за седмица
            week_metrics = {'week': str(week), 'n_matches': len(week_data)}
            
            # OU2.5 метрики
            if 'pred_ou25_prob' in week_data.columns:
                ou25_metrics = evaluate_calibration(
                    week_data['actual_ou25'].values,
                    week_data['pred_ou25_prob'].values
                )
                week_metrics['ou25_ece'] = ou25_metrics['ece']
                week_metrics['ou25_brier'] = ou25_metrics['brier_score']
            
            # BTTS метрики
            if 'pred_btts_prob' in week_data.columns:
                btts_metrics = evaluate_calibration(
                    week_data['actual_btts'].values,
                    week_data['pred_btts_prob'].values
                )
                week_metrics['btts_ece'] = btts_metrics['ece']
                week_metrics['btts_brier'] = btts_metrics['brier_score']
            
            weekly_metrics.append(week_metrics)
    
    if weekly_metrics:
        weekly_df = pd.DataFrame(weekly_metrics)
        
        # Интерактивен plot с Plotly
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=['OU2.5 ECE Trend', 'OU2.5 Brier Trend', 'BTTS ECE Trend', 'BTTS Brier Trend'],
            vertical_spacing=0.1
        )
        
        # OU2.5 ECE
        if 'ou25_ece' in weekly_df.columns:
            fig.add_trace(
                go.Scatter(x=weekly_df['week'], y=weekly_df['ou25_ece'], 
                          mode='lines+markers', name='OU2.5 ECE',
                          line=dict(color='blue')),
                row=1, col=1
            )
            fig.add_hline(y=0.07, line_dash="dash", line_color="red", 
                         annotation_text="Threshold", row=1, col=1)
        
        # OU2.5 Brier
        if 'ou25_brier' in weekly_df.columns:
            fig.add_trace(
                go.Scatter(x=weekly_df['week'], y=weekly_df['ou25_brier'], 
                          mode='lines+markers', name='OU2.5 Brier',
                          line=dict(color='green')),
                row=1, col=2
            )
            fig.add_hline(y=0.20, line_dash="dash", line_color="red", 
                         annotation_text="Threshold", row=1, col=2)
        
        # BTTS ECE
        if 'btts_ece' in weekly_df.columns:
            fig.add_trace(
                go.Scatter(x=weekly_df['week'], y=weekly_df['btts_ece'], 
                          mode='lines+markers', name='BTTS ECE',
                          line=dict(color='orange')),
                row=2, col=1
            )
            fig.add_hline(y=0.07, line_dash="dash", line_color="red", 
                         annotation_text="Threshold", row=2, col=1)
        
        # BTTS Brier
        if 'btts_brier' in weekly_df.columns:
            fig.add_trace(
                go.Scatter(x=weekly_df['week'], y=weekly_df['btts_brier'], 
                          mode='lines+markers', name='BTTS Brier',
                          line=dict(color='purple')),
                row=2, col=2
            )
            fig.add_hline(y=0.20, line_dash="dash", line_color="red", 
                         annotation_text="Threshold", row=2, col=2)
        
        fig.update_layout(
            title="📈 Weekly Calibration Trends",
            height=600,
            showlegend=False
        )
        
        fig.show()
        
        print(f"📊 Weekly trends calculated for {len(weekly_df)} weeks")
    else:
        print("❌ Not enough data for weekly trends")

## 🔧 Tuning History

In [ ]:
# Получава tuning история
tuning_history = tuner.get_tuning_history(days=30)

if 'error' not in tuning_history:
    print("🔧 TUNING HISTORY (Last 30 days)")
    print("=" * 50)
    
    summary = tuning_history['summary']
    print(f"📊 Summary:")
    print(f"   Total actions: {summary['total_actions']}")
    print(f"   Auto corrections: {summary['auto_corrections']}")
    print(f"   Manual adjustments: {summary['manual_adjustments']}")
    print()
    
    if tuning_history['history']:
        print("📋 Recent Actions:")
        for action in tuning_history['history'][-5:]:  # Последните 5
            timestamp = datetime.fromisoformat(action['timestamp']).strftime('%Y-%m-%d %H:%M')
            print(f"   {timestamp}: {action['action']} - {action['reason']}")
            
            if 'changes' in action and action['changes']:
                for param_group, changes in action['changes'].items():
                    if isinstance(changes, dict):
                        for param, change_info in changes.items():
                            if isinstance(change_info, dict) and 'old' in change_info:
                                print(f"     → {param_group}.{param}: {change_info['old']:.3f} → {change_info['new']:.3f}")
    else:
        print("✅ No tuning actions in the last 30 days")
else:
    print(f"❌ Error getting tuning history: {tuning_history['error']}")

## 🎯 Current Model Parameters

In [ ]:
# Показва текущите параметри
current_params = tuner.current_params

print("🎯 CURRENT MODEL PARAMETERS")
print("=" * 50)

print(f"🔧 Soft Clipping:")
print(f"   Lower bound: {current_params['soft_clipping']['lo']:.3f}")
print(f"   Upper bound: {current_params['soft_clipping']['hi']:.3f}")
print()

print(f"⚖️ Ensemble Weights:")
for model, weight in current_params['ensemble_weights'].items():
    print(f"   {model.capitalize()}: {weight:.3f}")
print()

print(f"🎲 Poisson Parameters:")
print(f"   Shrinkage alpha: {current_params['poisson_params']['shrinkage_alpha']:.3f}")
print(f"   Home advantage multiplier: {current_params['poisson_params']['home_advantage_multiplier']:.3f}")
print()

print(f"⚽ BTTS Calibration:")
print(f"   Scaling factor: {current_params['btts_calibration']['scaling_factor']:.3f}")
print(f"   Poisson blend: {current_params['btts_calibration']['poisson_blend']:.3f}")
print()

print(f"🤔 Confidence Parameters:")
print(f"   Entropy weight: {current_params['confidence_params']['entropy_weight']:.3f}")
print(f"   Agreement weight: {current_params['confidence_params']['agreement_weight']:.3f}")

## 🚨 Calibration Issues & Recommendations

In [ ]:
# Анализира текущите проблеми
analysis = tuner.analyze_calibration_drift(days=7)

if 'error' not in analysis:
    print("🚨 CURRENT CALIBRATION ANALYSIS")
    print("=" * 50)
    
    print(f"📊 Analysis period: {analysis['period']}")
    print(f"📈 Matches analyzed: {analysis['n_matches']}")
    print(f"⚠️ Issues detected: {len(analysis['issues_detected'])}")
    print()
    
    if analysis['issues_detected']:
        print("🚨 ISSUES FOUND:")
        for i, issue in enumerate(analysis['issues_detected'], 1):
            print(f"   {i}. {issue['market'].upper()}: {issue['issue'].replace('_', ' ').title()}")
            print(f"      Value: {issue['value']:.4f} (threshold: {issue['threshold']:.4f})")
            
            # Показва severity
            severity_ratio = issue['value'] / issue['threshold']
            if severity_ratio > 1.5:
                severity = "🔴 Critical"
            elif severity_ratio > 1.2:
                severity = "🟡 Moderate"
            else:
                severity = "🟢 Minor"
            print(f"      Severity: {severity} ({severity_ratio:.2f}x threshold)")
            print()
        
        print("💡 RECOMMENDATIONS:")
        for i, rec in enumerate(analysis['recommendations'], 1):
            print(f"   {i}. {rec['description']}")
            print(f"      Action: {rec['action']}")
            print(f"      Market: {rec['market']}")
            print()
    else:
        print("✅ No calibration issues detected!")
        print("   The model is well-calibrated within acceptable thresholds.")
else:
    print(f"❌ Error in calibration analysis: {analysis['error']}")

## 📋 Summary & Next Steps

In [ ]:
print("📋 CALIBRATION DASHBOARD SUMMARY")
print("=" * 60)

if len(df_with_results) > 0:
    print(f"📊 Data Status: ✅ {len(df_with_results)} matches with results")
    
    if 'error' not in report:
        # Оценява общото състояние
        issues_count = 0
        if 'ou25' in report and report['ou25']['ece'] > 0.07:
            issues_count += 1
        if 'btts' in report and report['btts']['ece'] > 0.07:
            issues_count += 1
        if '1x2' in report and 'overall' in report['1x2'] and report['1x2']['overall']['brier_score'] > 0.20:
            issues_count += 1
        
        if issues_count == 0:
            status = "🟢 EXCELLENT"
        elif issues_count <= 1:
            status = "🟡 GOOD"
        else:
            status = "🔴 NEEDS ATTENTION"
        
        print(f"🎯 Calibration Status: {status}")
        print(f"⚠️ Issues detected: {issues_count}")
    else:
        print(f"📊 Data Status: ❌ Error generating calibration report")
else:
    print(f"📊 Data Status: ❌ No prediction data available")

print(f"🔧 Tuning Status: {summary['total_actions']} actions in last 30 days")
print(f"📅 Last updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n🚀 NEXT STEPS:")
if len(df_with_results) == 0:
    print("   1. Ensure predictions are being logged properly")
    print("   2. Add match results to enable calibration analysis")
elif 'error' in analysis or analysis['n_matches'] < 100:
    print("   1. Collect more prediction data (need 100+ matches)")
    print("   2. Ensure match results are being updated")
elif analysis['issues_detected']:
    print("   1. Consider applying recommended corrections")
    print("   2. Monitor calibration trends closely")
    print("   3. Run daily monitoring script")
else:
    print("   1. Continue regular monitoring")
    print("   2. Maintain current parameter settings")
    print("   3. Review weekly calibration reports")

print("\n🎉 Dashboard analysis complete!")